In [144]:
import pandas as pd
from datetime import datetime
df1 = pd.concat(pd.read_excel('PV_firstRealease.xlsx', sheet_name=None), ignore_index=True)
df1.head()

,Hour,Starting minute (inclussive),Ending minute (exclussive),Generated power,TemperatureC,DewpointC,PressurehPa,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,dailyrainMM,SolarRadiationWatts_m2
0,0,0,5,0,0,0,0,0,0,0,0,0,0,0
1,0,5,10,0,0,0,0,0,0,0,0,0,0,0
2,0,10,15,0,11,8,1021,114,1,17,83,0,0,0
3,0,15,20,0,11,8,1021,134,12,17,83,0,0,0
4,0,20,25,0,11,8,1021,128,8,17,83,0,0,0


In [145]:
def remove_dataset_ouliers(dataset):
        list_outliers_index=[]
        for c in dataset:
            if type(c[0])!=str:
                list_outliers_index.extend(column_outliers(dataset, c))
        cleaned_dataset=remove_outliers(dataset, list_outliers_index)
        return cleaned_dataset

def column_outliers(df,c):
    q1=df[c].quantile(0.25)
    q3=df[c].quantile(0.75)
    iqr=q3-q1
    lower_bound=q1-4*iqr
    upper_bound=q3+4*iqr
    o = df.index[(df[c]<lower_bound) | (df[c]>upper_bound)]
    return o

def remove_outliers(df, outliers):
    outliers=sorted(set(outliers))
    df=df.drop(outliers)
    return df

In [157]:
df=df1
df["DateTime"] = df['Hour'].astype(str) +":"+ df["Starting minute (inclussive)"].astype(str)
df=df.drop(columns='Hour')
df=df.drop(columns='Ending minute (exclussive)')
df=df.drop(columns='Starting minute (inclussive)')

# set the initial date to '1989-12-31' and convert the time column to a datetime column
date='1989-12-31'
# convert the datetime strings to datetime objects
datetime_object= pd.to_datetime(date, format='%Y-%m-%d').date()
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%H:%M')
startDate=df['DateTime'][0].time()
firstTime=True
dates=df['DateTime']
for index in df.index:
    if dates[index].time()==startDate:
        datetime_object=datetime_object+ pd.Timedelta(days=1)
        date_entry=df['DateTime'][index].time()
        print(datetime_object, date_entry)
        new_datetime_entry=str(datetime_object)+" "+str(date_entry)
        print(new_datetime_entry)
        new_datetime_entry= pd.to_datetime(new_datetime_entry, format='%Y-%m-%d %H:%M')
        df.loc[index,"DateTime"]=new_datetime_entry
    else:
        date_entry=df['DateTime'][index].time()
        new_datetime_entry=str(datetime_object)+" "+str(date_entry)
        new_datetime_entry= pd.to_datetime(new_datetime_entry, format='%Y-%m-%d %H:%M')
        df.loc[index,"DateTime"]=new_datetime_entry
# df['DateTime'] = pd.to_timedelta(df['DateTime'])
# df["DateTime"] = df["DateTime"].dt.strftime("%H:%M")
# print(type(df["DateTime"][0]))
first_column=df.pop("DateTime")
df.insert(0, "DateTime", first_column)
# get the index
# idx = df.index
# df = df.rename_axis('index')
# df
# df.set_index("DateTime",append=True, inplace=True)
# # # print(df.columns.to_list())
# df=df.resample('15min',level='DateTime').mean()
# # # df
# df.reset_index(inplace=True)
# # df
# df['DateTime'] = pd.to_datetime(df['DateTime'], format='%H:%M').dt.time
df.head(96)

,DateTime,Generated power,TemperatureC,DewpointC,PressurehPa,WindDirectionDegrees,WindSpeedKMH,WindSpeedGustKMH,Humidity,HourlyPrecipMM,dailyrainMM,SolarRadiationWatts_m2
0,1990-01-01,0,0,0,0,0,0,0,0,0,0,0
1,1990-01-01,0,0,0,0,0,0,0,0,0,0,0
2,1990-01-01,0,11,8,1021,114,1,17,83,0,0,0
3,1990-01-01,0,11,8,1021,134,12,17,83,0,0,0
4,1990-01-01,0,11,8,1021,128,8,17,83,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
91,1990-01-01,0,13,11,1015,136,20,33,89,0,0,0
92,1990-01-01,0,13,11,1015,184,4,33,88,0,0,0
93,1990-01-01,0,13,11,1015,148,20,35,88,0,0,0
94,1990-01-01,0,13,11,1015,159,20,32,89,0,0,0


In [152]:
df.iloc[-1]
# df1.head(12)
# df_no_out=remove_dataset_ouliers(df)
# print('Before', len(df), 'After', len(df_no_out), 'Difference',(len(df)-len(df_no_out)))

DateTime                  1900-01-01 23:55:00
Day                                1992-12-30
Generated power                             0
TemperatureC                                8
DewpointC                                   4
PressurehPa                              1030
WindDirectionDegrees                      136
WindSpeedKMH                                3
WindSpeedGustKMH                            6
Humidity                                   75
HourlyPrecipMM                              0
dailyrainMM                                 0
SolarRadiationWatts_m2                      0
Name: 315359, dtype: object

In [158]:
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%H:%M').dt.time
df.head()

ValueError: time data '1990-01-01' does not match format '%H:%M' (match)